In [1]:
import sys
import os

# Agregar el directorio raíz del proyecto a sys.path
project_root = "/home/javitrucas/TFG"
if project_root not in sys.path:
    sys.path.append(project_root)

import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
import wandb
from box import Box
from types import SimpleNamespace
from scripts.dataset_loader import load_dataset
from scripts.MIL_utils import MIL_collate_fn

/home/javitrucas/miniconda3/envs/tfg/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/javitrucas/miniconda3/envs/tfg/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
# Configuración general
SAVE = True
SAVE_PATH = './resultados/panda_heatmaps/'
FIGSIZE = (10, 10)
SAVE_EXTENSION = 'png'

# Crear directorio si no existe
os.makedirs(SAVE_PATH, exist_ok=True)

# Configuración del dataset
SIZE = 512
RESIZE_SIZE = 256
DATA_DIR = f'/data/data_fran/Panda/patches_{SIZE}'

In [3]:
# Configuración para cargar el dataset
config = Box({
    "dataset_name": "panda-patches_512_preset-features_UNI",
    "input_feature_dim": 1024,
    "feature_dim": 128,
    "pooling_type": "attention",
    "batch_size": 1,
    "val_prop": 0.2,
    "seed": 42,
    "use_inst_distances": False,
    "adj_mat_mode": "relative"
})

# Cargar dataset de test
test_dataset = load_dataset(config=config, mode="test")

# Función para normalizar valores
def normalize(x):
    return (x - np.min(x)) / (np.max(x) - np.min(x))

# Obtener longitudes de bolsas
bag_len_list = []
for idx in range(len(test_dataset)):
    X, T, y, edge_index = test_dataset[idx]
    bag_len_list.append(len(X))

panda
[WSIDataset] Scanning files...


[WSIDataset] Building data dict: 100%|██████████| 1794/1794 [00:26<00:00, 68.08it/s]


[WSIDataset] Skipped 0 bags
[WSIDataset] Found 1794 already processed bags


In [4]:
# Ordenar índices por tamaño de bolsa
idx_bag_names_sorted = np.argsort(bag_len_list)

# Extraer índices de bolsas positivas
idx_pos_bags_sorted = []
for idx in idx_bag_names_sorted:
    X, T, y, edge_index = test_dataset[idx]
    
    # Asumiendo que consideramos un caso positivo si al menos un parche está etiquetado como 1
    if 1 in y:
        idx_pos_bags_sorted.append(idx)
    
print(f"Casos positivos encontrados: {len(idx_pos_bags_sorted)}")

# Seleccionar un caso positivo
if len(idx_pos_bags_sorted) > 0:
    BAG_IDX = idx_pos_bags_sorted[5] if len(idx_pos_bags_sorted) > 5 else idx_pos_bags_sorted[0]
    X, T, y, edge_index = test_dataset[BAG_IDX]
    adj_mat = edge_index.to_dense()

    print(f"Caso seleccionado: {BAG_IDX}")
    print(f"Número de parches: {len(X)}")
    
    # Mostrar información sobre las etiquetas
    num_positive = (y == 1).sum().item()
    num_negative = (y == 0).sum().item()
    print(f"Etiquetas: {num_positive} positivos, {num_negative} negativos")
    
    # Si quieres mostrar las etiquetas completas (podría ser muy largo)
    # print(f"Etiquetas de parches: {y}")
else:
    print("No se encontraron casos positivos.")

Casos positivos encontrados: 587
Caso seleccionado: 1551
Número de parches: 52
Etiquetas: 8 positivos, 44 negativos


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Parámetros del dataset
SIZE = 512
DATA_DIR = f'/data/data_fran/Panda/patches_{SIZE}'
csv_path = '/data/datasets/PANDA/PANDA_original/original/wsi_labels.csv'

# Leer el CSV
df = pd.read_csv(csv_path)

# Mostrar las columnas disponibles para verificar
print("Columnas disponibles en el CSV:")
print(df.columns.tolist())

# Seleccionar un índice específico de bolsa / slide
# Veamos un ejemplo con el primer image_id disponible
BAG_IDX = df['image_id'].iloc[0]  # Puedes cambiarlo por el ID deseado
print(f"Analizando WSI con ID: {BAG_IDX}")

# Filtrar los parches pertenecientes a esa WSI
wsi_metadata = df[df['image_id'] == BAG_IDX]

# Verificar si hay datos
if wsi_metadata.empty:
    print(f"No se encontraron datos para el WSI con ID: {BAG_IDX}")
else:
    print(f"Se encontraron {len(wsi_metadata)} parches para este WSI")

    # Comprobar si las columnas esperadas existen
    expected_cols = ['patch_id', 'row', 'col', 'isup_grade']
    available_cols = wsi_metadata.columns.tolist()
    
    # Mapear columnas si tienen nombres diferentes
    col_mapping = {}
    
    if 'patch_id' not in available_cols:
        # Intentar encontrar una columna alternativa para los parches
        if 'patch_name' in available_cols:
            col_mapping['patch_id'] = 'patch_name'
        else:
            print("No se encontró una columna para patch_id")
    
    if 'isup_grade' in available_cols:
        # Para el PANDA dataset, la etiqueta es el grado ISUP
        col_mapping['label'] = 'isup_grade'
    elif 'data_provider' in available_cols:
        # Si no hay isup_grade, podemos usar otra columna para demostración
        col_mapping['label'] = 'data_provider'
    
    # Aplicar mapeo de columnas
    for new_col, old_col in col_mapping.items():
        if old_col in available_cols:
            wsi_metadata[new_col] = wsi_metadata[old_col]
    
    # Extraer las coordenadas si están disponibles
    if 'row' in available_cols and 'col' in available_cols:
        # Listas con info útil
        if 'patch_id' in wsi_metadata.columns:
            PATCH_NAMES = wsi_metadata['patch_id'].tolist()
        else:
            PATCH_NAMES = [f"patch_{i}" for i in range(len(wsi_metadata))]
        
        if 'label' in wsi_metadata.columns:
            PATCH_LABELS = wsi_metadata['label'].tolist()
        else:
            PATCH_LABELS = [0] * len(wsi_metadata)
        
        row_list = wsi_metadata['row'].tolist()
        column_list = wsi_metadata['col'].tolist()

        # Normalizar coordenadas
        ROW_ARRAY = np.array(row_list)
        COL_ARRAY = np.array(column_list)
        min_row, min_col = ROW_ARRAY.min(), COL_ARRAY.min()
        ROW_ARRAY = ROW_ARRAY - min_row + 1
        COL_ARRAY = COL_ARRAY - min_col + 1

        # Tamaño del grid para la WSI
        MAX_ROW = int(ROW_ARRAY.max())
        MAX_COL = int(COL_ARRAY.max())
        print(f"Dimensiones normalizadas: {MAX_ROW} x {MAX_COL}")

        # Crear una matriz para visualizar los parches
        grid = np.zeros((MAX_ROW, MAX_COL))
        
        # Llenar la matriz con los valores de etiqueta
        for i in range(len(PATCH_LABELS)):
            row_idx = int(ROW_ARRAY[i]) - 1  # Índices basados en 0
            col_idx = int(COL_ARRAY[i]) - 1
            
            # Verificar que estamos dentro de los límites
            if 0 <= row_idx < MAX_ROW and 0 <= col_idx < MAX_COL:
                # Convertir etiquetas a valores numéricos si no lo son
                if isinstance(PATCH_LABELS[i], (int, float)):
                    grid[row_idx, col_idx] = PATCH_LABELS[i]
                else:
                    # Si es un string, asignar un valor numérico para visualización
                    grid[row_idx, col_idx] = hash(PATCH_LABELS[i]) % 10
        
        # Visualizar la matriz
        plt.figure(figsize=(12, 10))
        plt.imshow(grid, cmap='viridis')
        plt.colorbar(label='Etiqueta')
        plt.title(f'Visualización de WSI: {BAG_IDX}')
        plt.xlabel('Columna')
        plt.ylabel('Fila')
        plt.grid(False)
        plt.tight_layout()
        plt.savefig(f'wsi_visualization_{BAG_IDX}.png')
        plt.show()
        
        # Mostrar estadísticas de las etiquetas
        unique_labels = set(PATCH_LABELS)
        print(f"Etiquetas únicas: {unique_labels}")
        for label in unique_labels:
            count = PATCH_LABELS.count(label)
            print(f"  Etiqueta {label}: {count} parches ({count/len(PATCH_LABELS)*100:.1f}%)")
    else:
        print("No se encontraron columnas 'row' y 'col' para la visualización de coordenadas")
        
        # Mostrar un resumen de los datos disponibles
        print("\nResumen de datos disponibles:")
        print(wsi_metadata.describe())

Columnas disponibles en el CSV:
['image_id', 'data_provider', 'isup_grade', 'gleason_score', 'Partition']
Analizando WSI con ID: 0005f7aaab2800f6170c399693a96917
Se encontraron 1 parches para este WSI
No se encontró una columna para patch_id
No se encontraron columnas 'row' y 'col' para la visualización de coordenadas

Resumen de datos disponibles:
       isup_grade  label
count         1.0    1.0
mean          0.0    0.0
std           NaN    NaN
min           0.0    0.0
25%           0.0    0.0
50%           0.0    0.0
75%           0.0    0.0
max           0.0    0.0


/tmp/ipykernel_2922663/4000169243.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wsi_metadata[new_col] = wsi_metadata[old_col]


In [6]:
# Cargar las imágenes de los parches
patches_list = []
pbar = tqdm(total=len(PATCH_NAMES))
for patch_name in PATCH_NAMES:
    pbar.update(1)
    # Ajusta la ruta según tu estructura
    img = cv2.imread(f'{DATA_DIR}/images/{patch_name}.jpg')
    if img is None:  # Intenta con extensión alternativa si falla
        img = cv2.imread(f'{DATA_DIR}/images/{patch_name}.png')
    
    if img is not None:
        if RESIZE_SIZE != SIZE:
            img = cv2.resize(img, (RESIZE_SIZE, RESIZE_SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        patches_list.append(img)
    else:
        print(f"No se pudo cargar la imagen: {patch_name}")
        # Crear una imagen en blanco como fallback
        patches_list.append(np.ones((RESIZE_SIZE, RESIZE_SIZE, 3), dtype=np.uint8) * 255)

# Crear canvas para la WSI completa
MAX_X = (MAX_COL+2) * RESIZE_SIZE
MAX_Y = (MAX_ROW+2) * RESIZE_SIZE
canvas_wsi = np.ones((MAX_Y, MAX_X, 3), dtype=np.uint8) * 255

# Colocar parches en el canvas
for i, patch in enumerate(patches_list):
    row = ROW_ARRAY[i]
    column = COL_ARRAY[i]
    x = column * RESIZE_SIZE
    y = row * RESIZE_SIZE
    canvas_wsi[y:y+RESIZE_SIZE, x:x+RESIZE_SIZE] = patch

NameError: name 'PATCH_NAMES' is not defined

In [ ]:
def plot_wsi_and_heatmap(ax, wsi, heatmap=None, size=RESIZE_SIZE, plot_patch_contour=True, 
                          row_array=None, col_array=None, alpha=0.5):
    """
    Visualiza una WSI con su heatmap superpuesto.
    
    Args:
        ax: Matplotlib axis
        wsi: Canvas con la imagen WSI
        heatmap: Valores del heatmap (opcional)
        size: Tamaño de cada parche
        plot_patch_contour: Si se dibujan o no los contornos de los parches
        row_array: Array con las coordenadas de fila de cada parche
        col_array: Array con las coordenadas de columna de cada parche
        alpha: Transparencia del heatmap
    """
    ax.imshow(wsi)
    
    # Si se proporciona un heatmap, superponerlo
    if heatmap is not None:
        # Crear matriz para el heatmap
        if isinstance(heatmap, list):
            heatmap = np.array(heatmap)
        
        # Crear un mapa de calor vacío del tamaño del canvas
        hm = np.zeros((wsi.shape[0], wsi.shape[1]))
        
        # Llenar el mapa de calor con los valores
        for i in range(len(row_array)):
            row = row_array[i]
            col = col_array[i]
            y = row * size
            x = col * size
            value = heatmap[i]
            hm[y:y+size, x:x+size] = value
        
        # Crear una colormap personalizada
        import matplotlib.cm as cm
        from matplotlib.colors import ListedColormap
        
        N = 256
        vals = np.ones((N, 4))
        vals[:, 0] = np.linspace(0.17254901960784313, 0.8392156862745098, N)
        vals[:, 1] = np.linspace(0.6274509803921569, 0.15294117647058825, N)
        vals[:, 2] = np.linspace(0.17254901960784313, 0.1568627450980392, N)
        cmap = ListedColormap(vals)
        
        # Mostrar el heatmap
        ax.imshow(hm, alpha=alpha, cmap=cmap, vmin=0, vmax=1)
    
    # Dibujar contornos de los parches si se solicita
    if plot_patch_contour:
        for i in range(len(row_array)):
            row = row_array[i]
            col = col_array[i]
            y = row * size
            x = col * size
            rect = plt.Rectangle((x, y), size, size, fill=False, edgecolor='black', linewidth=0.5)
            ax.add_patch(rect)
    
    ax.set_xticks([])
    ax.set_yticks([])
    
    return ax

In [ ]:
# Visualizar WSI con y sin etiquetas
fig, ax = plt.subplots(figsize=(10,10), nrows=2)
ax[0] = plot_wsi_and_heatmap(ax[0], canvas_wsi, size=RESIZE_SIZE, 
                            plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[1] = plot_wsi_and_heatmap(ax[1], canvas_wsi, PATCH_LABELS, size=RESIZE_SIZE, 
                            plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[0].set_title("WSI Original")
ax[1].set_title("Etiquetas de parches")
plt.tight_layout()
plt.show()

# Guardar figuras
if SAVE:
    # Guardar WSI sin etiquetas
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax = plot_wsi_and_heatmap(ax, canvas_wsi, size=RESIZE_SIZE, 
                             plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
    plt.savefig(f'{SAVE_PATH}/panda_wsi_patched.{SAVE_EXTENSION}', bbox_inches='tight')
    
    # Guardar WSI con etiquetas
    fig, ax = plt.subplots(figsize=FIGSIZE)
    ax = plot_wsi_and_heatmap(ax, canvas_wsi, PATCH_LABELS, size=RESIZE_SIZE, 
                             plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
    plt.savefig(f'{SAVE_PATH}/panda_wsi_patched_labels.{SAVE_EXTENSION}', bbox_inches='tight')

In [19]:
# Suavizar etiquetas basado en los vecinos
changes = False
old_patch_labels = PATCH_LABELS.copy()

while True:
    new_patch_labels = []
    changes = False
    
    for i in range(len(old_patch_labels)):
        sum_pos = 0
        sum_neg = 0
        for j in range(len(old_patch_labels)):
            if adj_mat[i, j] > 0:
                if old_patch_labels[j] == 1:
                    sum_pos += 1
                else:
                    sum_neg += 1
         
        if sum_pos > sum_neg:
            new_label = 1
        else:
            new_label = 0
        new_patch_labels.append(new_label)
        if new_label != old_patch_labels[i]:
            changes = True
    
    if not changes:
        break
    old_patch_labels = new_patch_labels.copy()

print("Suavizado de etiquetas finalizado")

# Visualizar resultados del suavizado
fig, ax = plt.subplots(figsize=(10,10), ncols=2)
ax[0] = plot_wsi_and_heatmap(ax[0], canvas_wsi, PATCH_LABELS, size=RESIZE_SIZE, 
                            plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[1] = plot_wsi_and_heatmap(ax[1], canvas_wsi, new_patch_labels, size=RESIZE_SIZE, 
                            plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[0].set_title("Etiquetas originales")
ax[1].set_title("Etiquetas suavizadas")
plt.tight_layout()
plt.show()

# Actualizar etiquetas
PATCH_LABELS = new_patch_labels

NameError: name 'PATCH_LABELS' is not defined

In [ ]:
# Diccionario de rutas de modelos entrenados
# (Aquí debes usar las rutas de tus modelos)
run_ids = {
    'attention': 'tu_run_id_attention',
    'mean': 'tu_run_id_mean',
    'max': 'tu_run_id_max'
}

# Configuración para cargar modelos
from scripts.medical_scripts.medical_evaluation import ModelEvaluator
from scripts.MIL_utils import MIL_collate_fn

# Preparar datos para predicción
X_batch = X.unsqueeze(0)  # Añadir dimensión de batch
T_batch = T.unsqueeze(0) if T is not None else None
y_batch = y.unsqueeze(0)
edge_index_batch = edge_index.unsqueeze(0) if edge_index is not None else None
mask = torch.ones_like(y_batch)

# Diccionario para almacenar predicciones
f_pred_dict = {}

# Cargar y aplicar cada modelo
for model_name, run_id in run_ids.items():
    print(f"Procesando modelo: {model_name}")
    
    # Cargar configuración del modelo desde wandb
    api = wandb.Api()
    try:
        run = api.run(f"tu_usuario/TFG/{run_id}")
        config_dict = run.config
        
        # Crear configuración para evaluador
        config = Box({
            "dataset_name": config_dict["dataset_name"],
            "input_feature_dim": config_dict["input_feature_dim"],
            "feature_dim": config_dict["feature_dim"],
            "pooling_type": model_name,
            "batch_size": 1
        })
        
        # Ruta al modelo guardado
        model_path = f"./models/{config.dataset_name.split('-')[0]}/{model_name}/model.pth"
        
        # Crear el evaluador
        evaluator = ModelEvaluator(
            model_path=model_path,
            test_loader=None,
            batch_size=1,
            input_feature_dim=config.input_feature_dim,
            feature_dim=config.feature_dim,
            pooling_type=model_name,
            wandb=None
        )
        
        # Obtener el modelo y ponerlo en modo evaluación
        model = evaluator.model
        model.eval()
        
        # Predecir atenciones
        with torch.no_grad():
            # Esto dependerá de la estructura de tu modelo
            # Ajusta estas líneas según tu implementación
            if hasattr(model, 'predict_attentions'):
                _, attn = model.predict_attentions(X_batch, edge_index_batch, mask)
                f_pred = attn.squeeze(0).cpu().numpy()
            else:
                # Función alternativa para extraer atenciones
                # Por ejemplo, con un forward hook para capturar el mapa de atención
                attn_maps = []
                
                def hook_fn(module, input, output):
                    attn_maps.append(output)
                
                # Registrar hook en la capa de atención
                if hasattr(model, 'attention'):
                    hook = model.attention.register_forward_hook(hook_fn)
                    
                _ = model(X_batch, edge_index_batch, mask)
                
                if attn_maps:
                    f_pred = attn_maps[0].squeeze(0).cpu().numpy()
                    hook.remove()
                else:
                    # Si no se puede extraer la atención, usa una distribución uniforme
                    f_pred = np.ones(len(X)) / len(X)
        
        # Normalizar predicciones
        f_pred = normalize(f_pred)
        f_pred_dict[model_name] = f_pred
        
    except Exception as e:
        print(f"Error al cargar el modelo {model_name}: {e}")
        f_pred_dict[model_name] = np.ones(len(X)) / len(X)  # Atención uniforme como fallback

In [ ]:
# Visualizar mapas de atención de todos los modelos
fig, ax = plt.subplots(figsize=(15,10), ncols=len(f_pred_dict))
if len(f_pred_dict) == 1:
    ax = [ax]  # Convertir a lista si solo hay un modelo

for i, (model_name, f_pred) in enumerate(f_pred_dict.items()):
    ax[i] = plot_wsi_and_heatmap(ax[i], canvas_wsi, f_pred, size=RESIZE_SIZE, 
                                plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
    ax[i].set_title(model_name)

plt.tight_layout()
plt.show()

# Guardar mapas de atención individuales
if SAVE:
    for model_name, f_pred in f_pred_dict.items():
        fig, ax = plt.subplots(figsize=FIGSIZE)
        ax = plot_wsi_and_heatmap(ax, canvas_wsi, f_pred, size=RESIZE_SIZE, 
                                 plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
        ax.set_title(model_name)
        
        # Añadir barra de color
        from matplotlib.colors import ListedColormap
        import matplotlib.cm as cm
        from mpl_toolkits.axes_grid1.inset_locator import inset_axes
        
        N = 256
        vals = np.ones((N, 4))
        vals[:, 0] = np.linspace(0.17254901960784313, 0.8392156862745098, N)
        vals[:, 1] = np.linspace(0.6274509803921569, 0.15294117647058825, N)
        vals[:, 2] = np.linspace(0.17254901960784313, 0.1568627450980392, N)
        cmap = ListedColormap(vals)
        
        cbaxes = inset_axes(ax, width="30%", height="3%", loc='upper right') 
        fig.colorbar(
            cm.ScalarMappable(norm=None, cmap=cmap), 
            cax=cbaxes, 
            orientation='horizontal',
            ticks=[0,1]
        )
        
        plt.savefig(f'{SAVE_PATH}/panda_wsi_attention_{model_name}.{SAVE_EXTENSION}', bbox_inches='tight')

In [ ]:
# Visualizar comparativa entre etiquetas de parches y mejor modelo de atención
# Seleccionar el modelo con mejor rendimiento (puedes ajustar este criterio)
best_model = list(f_pred_dict.keys())[0]  # Por defecto, usar el primero

fig, ax = plt.subplots(figsize=(15,5), ncols=3)
ax[0] = plot_wsi_and_heatmap(ax[0], canvas_wsi, size=RESIZE_SIZE, 
                            plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[1] = plot_wsi_and_heatmap(ax[1], canvas_wsi, PATCH_LABELS, size=RESIZE_SIZE, 
                            plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)
ax[2] = plot_wsi_and_heatmap(ax[2], canvas_wsi, f_pred_dict[best_model], size=RESIZE_SIZE, 
                            plot_patch_contour=True, row_array=ROW_ARRAY, col_array=COL_ARRAY)

ax[0].set_title("WSI Original")
ax[1].set_title("Etiquetas de parches")
ax[2].set_title(f"Atención ({best_model})")

plt.tight_layout()
plt.show()

if SAVE:
    plt.savefig(f'{SAVE_PATH}/panda_wsi_comparison.{SAVE_EXTENSION}', bbox_inches='tight')

In [ ]:
# Análisis cuantitativo - correlación entre etiquetas y mapas de atención
correlations = {}
for model_name, f_pred in f_pred_dict.items():
    # Correlación de Pearson
    corr = np.corrcoef(np.array(PATCH_LABELS), f_pred)[0,1]
    correlations[model_name] = corr
    print(f"Correlación con {model_name}: {corr:.4f}")

# Visualizar correlaciones
plt.figure(figsize=(8,5))
plt.bar(correlations.keys(), correlations.values())
plt.xlabel("Modelo")
plt.ylabel("Correlación con etiquetas")
plt.title("Correlación entre mapas de atención y etiquetas de parches")
plt.ylim(0, 1)  # Ajusta según tus resultados
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

if SAVE:
    plt.savefig(f'{SAVE_PATH}/correlations.{SAVE_EXTENSION}', bbox_inches='tight')